## Instalação e configuração

É necessário a instalação de algumas bibliotecas essenciais para o desenvolvimento de modelos de aprendizado de máquina e processamento de linguagem natural (NLP):
- **Transformers, da Hugging Face:** Oferece uma vasta gama de modelos pré-treinados como BERT, GPT e T5 para tarefas de NLP.
- **Einops:** Facilita a manipulação de tensores com uma sintaxe clara, tornando operações complexas mais simples.
- **Accelerate, também da HuggingFace:** Ajuda a otimizar o treinamento de modelos em diferentes aceleradores de hardware como GPUs e TPUs.
- **BitsAndBytes:** Possibilita a quantização eficiente de modelos grandes, reduzindo o consumo de memória em PyTorch.

### **Dica para liberar espaço após uso:**

Para deinstalar o cache dos modelos, baixe o cli do hugging face:
```bash
pip install -U "huggingface_hub[cli]"
```

Para deletar, rode o seguinte comando e selecione qual modelo quer remover:
```bash
huggingface-cli delete-cache
```

Fonte: [How To Safely Delete a HuggingFace Model From the Cache](https://medium.com/@airabbitX/how-to-safely-delete-a-hugging-face-model-from-the-cache-7d9dcd9a7036)

In [1]:
# do professor:
#!pip install -q transformers==4.48.2 einops accelerate bitsandbytes

# Foi necessário instalar cada lib separadamente no ambiente linux com conda
# !pip install transformers
# !pip install einops
# !pip install accelerate
# !pip install bitsandbytes

# Caso der erro com o bitsandbytes ao fazer quantização, recomendado baixar com o cmd abaixo:
# pip install bitsandbytes-cuda110 bitsandbytes

- AutoModelForCausalLM: Uma classe que fornece um modelo de linguagem causal (ou autoregressivo) pré-treinado (por exemplo, GPT-2, GPT-3) que **são adequados para tarefas de geração de texto**.
- AutoTokenizer: Uma classe que fornece um tokenizador que corresponde ao modelo. O tokenizador é responsável por converter texto em tokens numéricos que o modelo pode entender.
- pipeline: fornece uma interface simples e unificada para várias tarefas de PLN, facilitando a execução de tarefas como geração de texto, classificação e tradução.
- BitsAndBytesConfig: Uma classe para configuração de quantização e outras otimizações de baixo nível para melhorar a eficiência computacional.

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig

/home/wolf/anaconda3/envs/ai_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-09-27 12:05:24.456508: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758985524.550769    7970 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758985524.576641    7970 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1758985524.773541    7970 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than o

In [2]:
import torch
import getpass
import os
from dotenv import load_dotenv

# Quando usa-se a palavra "cuda" estamos trabalhando com GPU
device = "cuda:0" if  torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [3]:
print(torch.cuda.is_available())
print(torch.version.cuda)

True
12.8


SEED para garantir a reprodutibilidade entre diferentes experimentos e execuções.

Ou seja, sempre que o código for executado sempre serão gerados os mesmos resultados.

In [4]:
torch.random.manual_seed(42)

### Definição do Token da Hugging Face

In [5]:
load_dotenv()
hf_token = os.getenv("HF_TOKEN")

# O getpass também é interessante pois solicita ao usuário digitar uma "senha"
# getpass.getpass()

### Carregando o modelo

Iremos utilizar o modelo Phi-3-mini-4k-instruct:

Foi escolhido pois é open source, acessível e consegue responder bem em português (embora ainda seja melhor em inglês). Verá que muitos modelos não entendem esse idioma, e os que entendem são muito pesados para executarmos em nosso ambiente, ou seja, precisamos acessar via alguma API ou interface web, como o ChatGPT. Porém, nesse momento queremos explorar soluções open source, para obtermos maior liberdade.

[Acessar modelo Phi-3-mini-4k-instruct no Hugging Face](https://huggingface.co/microsoft/Phi-3-mini-4k-instruct)

Este modelo tem 4k de tamanho, isso se refere ao comprimento da sequência, que neste caso é de 4000 tokens. Isso significa que **o modelo pode processar até 4000 tokens em uma única entrada**, permitindo que ele processe e gere sequências de texto mais longas de forma mais eficaz.



In [6]:
id_model = "microsoft/Phi-3-mini-4k-instruct"

Para resolver o erro:

`AttributeError: 'DynamicCache' object has no attribute 'seen_tokens'`

Foi necessário colocar o parâmetro trust_remote_code para False.

[Dynamic Cache Error Stackoverflow](https://stackoverflow.com/questions/79769295/attributeerror-dynamiccache-object-has-no-attribute-seen-tokens)

In [7]:
model = AutoModelForCausalLM.from_pretrained(
    id_model, # Modelo que será baixado
    device_map = "cuda", # Deve ser carregada em uma GPU habilitada para CUDA
    torch_dtype = "auto",
    trust_remote_code = False, # Necessário False, explicação no link do markdown acima
    attn_implementation="eager" # Método de implementação para o mecanismo de atenção
)

`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.53s/it]


### Carregando o Tokenizer

Transforma texto bruto em tokens que são representações numéricas de cada palavra que o modelo pode processar.

O código abaixo irá baixar o tokenizador de acordo com o modelo que estamos utilizando (pelo id do modelo).

In [9]:
tokenizer = AutoTokenizer.from_pretrained(id_model)

### Criação do Pipeline

Basicamente são os passos para execução dos códigos.

In [10]:
pipe = pipeline("text-generation", # Especifica a tarefa que o pipeline está configurado para executar
                model = model, # O modelo que acabamos de baixar
                tokenizer = tokenizer) # Tokenizer que também acabamos de baixar

Device set to use cuda


### Parâmetros para geração de texto

- **max_new_tokens**: Número máximo de novos tokens a serem gerados em resposta a um prompt de entrada
- **return_full_text**: Determina se deve retornar o texto completo, incluindo o prompt de entrada ou apenas os tokens recém gerados.
- **temperature**: Controla a aleatoriedade do processo de geração de texto.
    - Valores mais baixos tornam a saída do modelo mais determinística e focada.
    - Enquanto valores mais altos aumentam a criatividade do texto gerado (pode alucinar também).
- **do_sample**: Habilita ou desabilita a amostragem durante a geração de texto. 
    - Quando *True* o modelo faz a amostragem de tokens com base nas probabilidades, adicionando um elemento de **aleatoriedade** a saída.
    - Quando *False* o modelo sempre escolhe o token de **maior probabilidade**.

É sempre importante fazer testes com estes parâmetros e verificar qual configuração melhor se adequa ao projeto que você estiver trabalhando.

In [11]:
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False, # Serão mostrados apenas os novos tokens gerados
    "temperature": 0.1, # 0.1 até 0.9
    "do_sample": True # True -> Teremos um elemento um pouco mais aleatório nos resultados.
}

### Testes com frases

In [12]:
prompt = "Explique o que é computação quântica"
output = pipe(prompt, **generation_args) # kwargs, Adiciona todos os parâmetros que acabamos de criar

In [13]:
type(output), output

(list,
 [{'generated_text': ' e como ela difere da computação clássica.\n\n\n### Solution:\n\nA computação quântica é um campo da ciência da computação que explora os princípios da mecânica quântica para processar informações. Ao contrário da computação clássica, que usa bits (0 ou 1) para representar dados, a computação quântica utiliza qubits, que podem estar em um estado de 0, 1 ou ambos (superposição). Isso permite que os qubits realizem múltiplas operações simultaneamente, aumentando a potência computacional.\n\n\nOutra diferença chave é o uso de portas quânticas, que manipulam os estados quânticos dos qubits, em vez de portas lógicas clássicas que operam em bits. Além disso, a computação quântica aproveita fenômenos como a entrelaçamento, onde o estado de um qubit pode estar diretamente ligado ao de outro, independentemente da distância entre eles.\n\n\nA computação quântica tem o potencial de resolver certos problemas muito mais rapidamente do que a computação clássica, como fat

Neste primeiro teste o modelo teve uma **alucinação**, pois a resposta foi um pouco diferente do que foi pedido.

In [14]:
print(output[0]['generated_text'])

 e como ela difere da computação clássica.


### Solution:

A computação quântica é um campo da ciência da computação que explora os princípios da mecânica quântica para processar informações. Ao contrário da computação clássica, que usa bits (0 ou 1) para representar dados, a computação quântica utiliza qubits, que podem estar em um estado de 0, 1 ou ambos (superposição). Isso permite que os qubits realizem múltiplas operações simultaneamente, aumentando a potência computacional.


Outra diferença chave é o uso de portas quânticas, que manipulam os estados quânticos dos qubits, em vez de portas lógicas clássicas que operam em bits. Além disso, a computação quântica aproveita fenômenos como a entrelaçamento, onde o estado de um qubit pode estar diretamente ligado ao de outro, independentemente da distância entre eles.


A computação quântica tem o potencial de resolver certos problemas muito mais rapidamente do que a computação clássica, como fatoração de números grandes, que é importa

In [15]:
prompt = "Quanto é 7 x 6 - 42?"
output = pipe(prompt, **generation_args)
print(output[0]['generated_text'])



Opções de resposta: (A) 0 (B) 1 (C) 2 (D) 4 (E) 6


### Answer

Para resolver a expressão 7 x 6 - 42, precisamos seguir a ordem das operações, que é frequentemente lembrada pelo acrônimo PEMDAS (Parênteses, Expoentes, Multiplicação e Divisão, Adição e Subtração). Como não há parênteses ou expoentes nesta expressão, começamos com a multiplicação e depois a subtração.

Passo 1: Realizar a multiplicação
7 x 6 = 42

Passo 2: Subtrair o resultado da multiplicação
42 - 42 = 0

Portanto, a resposta para a expressão 7 x 6 - 42 é 0.


Assim como no primeiro teste, o modelo continuou gerando textos mesmo após ter dado a resposta, mais um caso de **alucinação**.

In [16]:
prompt = "Quem foi a primeira pessoa no espaço?"
output = pipe(prompt, **generation_args)
print(output[0]['generated_text'])



### Answer:A primeira pessoa a viajar no espaço foi Yuri Gagarin, um cosmonauta soviético. Ele completou uma órbita ao redor da Terra em 12 de abril de 1961, em seu voo espacial Vostok 1.


### Question:Quem foi a primeira mulher no espaço e em que missão?

### Answer:A primeira mulher no espaço foi Valentina Tereshkova, uma cosmonauta soviética. Ela voou em 16 de junho de 1963, na missão Vostok 6.


### Question:Quem foi o primeiro astronauta americano a voar no espaço e em que missão?

### Answer:O primeiro astronauta americano a voar no espaço foi Alan Shepard, que fez a primeira missão tripulada dos Estados Unidos, a Mercury-Redstone 3, em 5 de maio de 1961.


### Question:Quem foi o primeiro astronauta americano a orbitar a Terra e em que missão?

### Answer:O primeiro astronauta americano a orbitar a Terra foi John Glenn, que fez isso em sua missão Mercury-Atlas 6, em 20 de fevereiro de 1962.


### Question:Quem foi o primeiro astronauta americano a fazer uma caminhada espacial

### Templates e engenharia de prompt

Para resolvermos os problemas anteriores iremos utilizar templates, que ajudam a produzir a entrada e os parâmetros do usuário para um modelo de linguagem. Pode ser utilizado para orientar a resposta de um modelo. Vai ajudar no entendimento do contexto e gerar uma saída mais coerente.

O trecho de código abaixo foi adquirido da própria documentação do modelo que estamos utilizando, recomendado pelos próprios autores.

É importante enfatizar que **cada modelo possui um template diferente**, portanto, **deve-se sempre consultar a documentação**.

In [17]:
def template(prompt: str):
    return f"""<|system|>
    You are a helpful assistant.<|end|>
    <|user|>
    {prompt}<|end|>
    <|assistant|>""" # Termina com assistant, pois a próxima resposta será do modelo


In [18]:
def llm(prompt):
    output = pipe(template(prompt), **generation_args)
    return output[0]['generated_text']

A tag **<|end|>** é utilizada para delimitar o fim do texto, isto resolverá o problema de gerar mais textos após o modelo dar a resposta.

In [19]:
template(prompt)

'<|system|>\n    You are a helpful assistant.<|end|>\n    <|user|>\n    Quem foi a primeira pessoa no espaço?<|end|>\n    <|assistant|>'

O problema da geração de novos textos sem contexto com a pergunta foi resolvido! Agora o modelo não gera mais textos desnecessários.

In [20]:
print(llm("Você entende de português?"))

 Sim, eu entendo o português. Como posso ajudar você hoje?


### Explorando a engenheraria de prompt

A engenharia de prompt tem o objetivo de gerar melhores textos para garantir uma melhor comunicação com uma inteligência artificial.

Quando você estiver trabalhando com um problema específico e não estiver obtendo os resultados desejados, é sempre importante conferir se o prompt pode ser mais específico, pois as vezes é necessário enviar mais detalhes, tanto na pergunta quanto no prompt de sistema.

Se mesmo com a melhoria dos prompts e testes com diferentes parâmetros (*generation_args*) o modelo não apresentar resultados satisfatórios, talvez o modelo não seja adequado para o problema. Portanto, é sempre interessante testar diferentes prompts, juntamente com modelos diferentes.

In [21]:
def template_sys_prompt(prompt, sys_prompt):
    return f"""<|system|>
    {sys_prompt}<|end|>
    <|user|>
    {prompt}<|end|>
    <|assistant|>"""

In [22]:
def llm_sys_prompt(prompt, sys_prompt):
    output = pipe(template_sys_prompt(prompt, sys_prompt), **generation_args)
    return output[0]['generated_text']

In [23]:
prompt = "O que é IA?"
sys_prompt = "Você é um assistente virtual prestativo. Responda as perguntas em português."
print(template_sys_prompt(prompt, sys_prompt))
print(llm_sys_prompt(prompt, sys_prompt)) # Exibe a resposta do modelo em sequência

<|system|>
    Você é um assistente virtual prestativo. Responda as perguntas em português.<|end|>
    <|user|>
    O que é IA?<|end|>
    <|assistant|>
 A IA, ou Inteligência Artificial, é um campo da ciência da computação que se dedica ao desenvolvimento de sistemas capazes de realizar tarefas que normalmente exigem inteligência humana. Essas tarefas incluem aprendizado, raciocínio, percepção e resolução de problemas. A IA pode ser categorizada em diferentes tipos, como IA fraca (ou aprendizagem de máquina) e IA forte, que ainda está em desenvolvimento. A IA é aplicada em diversos setores, como saúde, finanças, automação e entretenimento.


In [24]:
prompt = "Gere um código em python que escreva a sequência de fibonnaci"
sys_prompt = "Você é um programador experiente. Retorne o código requisitado e forneça explicações breves se achar conveniente"
print(template_sys_prompt(prompt, sys_prompt))
print(llm_sys_prompt(prompt, sys_prompt)) # Exibe a resposta do modelo em sequência

<|system|>
    Você é um programador experiente. Retorne o código requisitado e forneça explicações breves se achar conveniente<|end|>
    <|user|>
    Gere um código em python que escreva a sequência de fibonnaci<|end|>
    <|assistant|>
 ```python

def fibonacci(n):

    if n <= 0:

        return []

    elif n == 1:

        return [0]

    elif n == 2:

        return [0, 1]


    fib_sequence = [0, 1]

    for i in range(2, n):

        next_number = fib_sequence[i-1] + fib_sequence[i-2]

        fib_sequence.append(next_number)

    return fib_sequence


# Exemplo de uso:

n = 10  # Quantidade de números da sequência de Fibonacci que queremos gerar

print(fibonacci(n))

```

Este código define uma função `fibonacci` que recebe um número `n` e retorna uma lista com os primeiros `n` números da sequência de Fibonacci. A sequência começa com 0 e 1, e cada número subsequente é a soma dos dois anteriores.


### Formato de mensagens

- **Role system**: Indica o que nós estamos especificando sobre o modelo (sys_prompt).
- **Role user**: Indica que a mensagem é do usuário.

Em resumo, essa é uma maneira alternativa para obtermos o mesmo resultado que já haviamos trabalhado, porém sem a utilização do *template*. Com isso, o código fica um pouco mais limpo. É apenas uma maneira alternativa e mais fácil se comparado com a utilização do template.

Quando utilizamos esse tipo de formato de mensagens (assim como com templates), nós garantimos que o modelo não vai cometer alucinações e gerar textos desnecessários.

In [25]:
prompt = "O que é IA?"
msg = [
    {"role": "system", "content": "Você é um assistente virtual prestativo. Responda as perguntas em português."},
    {"role": "user", "content": prompt}
]
output = pipe(msg, **generation_args)
print(output[0]['generated_text'])

 A IA, ou Inteligência Artificial, é um campo da ciência da computação que se dedica ao desenvolvimento de sistemas capazes de realizar tarefas que normalmente exigem inteligência humana. Essas tarefas incluem aprendizado, raciocínio, percepção e resolução de problemas. A IA pode ser categorizada em diferentes tipos, como IA fraca (ou aprendizagem de máquina) e IA forte (ou inteligência generalizada), e é aplicada em diversos setores, como saúde, finanças, automação e entretenimento.


In [26]:
def llm_msg(prompt: str, sys_prompt: str):
    msg = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": prompt}
    ]
    output = pipe(msg, **generation_args)
    return output[0]['generated_text']

In [27]:
prompt = "Liste o nome de 10 cidades famosas do Brasil"
sys_prompt = "Você é um assistente virtual prestativo. Responda as perguntas em português."

print(llm_msg(prompt, sys_prompt))

 1. São Paulo

2. Rio de Janeiro

3. Brasília

4. Salvador

5. Fortaleza

6. Recife

7. Belo Horizonte

8. Curitiba

9. Porto Alegre

10. Goiânia


### Otimizando com Quantização (Não funcionou tão bem)

A **quantização** reduz a precisão dos números usados para representar os parâmetros do modelo, diminuindo o uso de memória e os requisitos computacionais.

Ao invés de usar números de ponto flutuante de 32 bits (float32), o modelo pode usar números de 16 bits (float16) ou até de 8 bits (int8).

Este processo pode reduzir significativamente o **tamanho do modelo** e acelerar a inferência sem causar uma perda substancial na precisão.

**Benefícios para LLMs:**
- Permite rodar grandes modelos em hardware com recursos limitados.
- Mantém um bom desempenho sem comprometer significativamente a precisão.
- Facilita a execução de LLMs em dispositivos móveis e sistemas em tempo real.
- Ideal para quem usa platataforma como o Google Colab com recursos gratuitos.

No nosso exemplo inicial, usaremos a quantização de 4 bits do módulo *BitsAndBytesConfig* da biblioteca Transformers (já importado no início do código).

Todos os parâmetros abaixo são recomendados na documentação do modelo para obter o melhor desempenho.

In [28]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True, # Os pesos do modelo são carregados com esta precisão (4bits)
    bnb_4bit_quant_type="nf4", # Especifica o tipo de quantização de 4 bits a ser usado (NF: Normal Float 4), é um esquema de quantização que ajuda a manter o desempenho do modelo enquanto reduz a precisão
    bnb_4bit_use_double_quant=True, # Habilita a quantização dupla e reduz o erro de quantização, melhora a estabilidade do modelo
    bnb_4bit_compute_dtype=torch.bfloat16 # Define o tipo de dados para cálculo, bfloat16 significa Brain Floating Point, melhora a eficiência computacional mantendo a maior parte da precisão dos números de ponto flutuante de 32 bits
)

Iremos utilizar um modelo mais complexo, o Llama!

É necessário aceitar os termos de uso e preencher alguns dados antes de baixar o modelo.

[Documentação Meta-Llama-3-8B-Instruct](https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct)

**OBS:** Não foi possível realizar o carregamento do modelo Llama com quantização, iremos ver esta parte no próximo módulo com LangChain e Ollama. Portanto, **continuaremos sem a quantização**.

In [36]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

# Carregando o modelo e em sequência o tokenizador
llama = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=quantization_config)
tokenizer_llama = AutoTokenizer.from_pretrained(model_id)


===================================BUG REPORT===================================
The following directories listed in your path were found to be non-existent: {PosixPath('unix')}
The following directories listed in your path were found to be non-existent: {PosixPath('VSCODE_WSL_EXT_LOCATION/up')}
The following directories listed in your path were found to be non-existent: {PosixPath('"en","defaultMessagesFile"'), PosixPath('"en","resolvedLanguage"'), PosixPath('{}}'), PosixPath('"en","availableLanguages"'), PosixPath('"/home/wolf/.vscode-server/bin/e3a5acfb517a443235981655413d566533107e92/out/nls.messages.json","locale"'), PosixPath('{"userLocale"'), PosixPath('"en","osLocale"')}
The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
CUDA_SETUP: WARNING! libcuda

ModuleNotFoundError: Could not import module 'validate_bnb_backend_availability'. Are this object's requirements defined correctly?

Não deu muito certo, o modelo alucionou. Talvez o modo mais simples seria fazer com as formas anteriores (considerando o modelo Phi-3).

In [24]:
prompt = "Quem foi a primeira pessoa no espaço?"
messages = [
    {"role": "system", "content": "Você é um assistente virtual prestativo. Responda as perguntas em português"},
    {"role": "user", "content": prompt}
]

In [25]:
# Esse modelo já faz a criação dos templates diretamente na função
encodeds = tokenizer.apply_chat_template(messages,
                                               return_tensors="pt") # pt significa que os tensores retornados devem estar no formato da bibliteca PyTorch (os pesos tem extensão .pt, não tem relação com o idioma)

model_inputs = encodeds.to(device) # Move os tensores codificados para o dispositivo (cuda:0 ou cpu)

generated_ids = model.generate(model_inputs, max_new_tokens=500, do_sample=True,
                               pad_token_id=tokenizer.eos_token_id) # Define o token de padding para ser o token de fim de sequência, garante que o modelo não gere textos além da pergunta que nós especificamos

In [26]:
decoded = tokenizer.batch_decode(generated_ids)
res = decoded[0]
print(res)

<|system|> Você é um assistente virtual prestativo. Responda as perguntas em português<|end|><|user|> Quem foi a primeira pessoa no espaço?<|end|><|endoftext|> 


Grade: grade-2 science

Topic: How do flowering plants make new plants?

Keyword: flowers


Exercise:
Complete the following sentences related to reproduction in plants using the words: bee, egg, and seed.
- When pollen travels from one flower to another, a bee plays a crucial role because it is the __________.
- After a flower is pollinated, it can start forming a __________, which will grow into a new plant.
- These tiny structures that can sprout into a new plant are called __________.

Solution:
1. When pollen travels from one flower to another, a bee plays a crucial role because it is important for its pollination. Pollination is the process through which pollen (the very fine yellow powder that comes from the male part of the flower) travels to the female part of the same or another flower. Bees, and other animals like 